# s_characteristic_port_rev [<img src="https://www.arpm.co/lab/icons/icon_permalink.png" width=30 height=30 style="display: inline;">](https://www.arpm.co/lab/redirect.php?code=s_characteristic_port_rev&codeLang=Python)
For details, see [here](https://www.arpm.co/lab/redirect.php?permalink=eb-mom-signal-copy-1).

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

from arpym.statistics.ewm_meancov import ewm_meancov
from arpym.statistics.meancov_sp import meancov_sp
from arpym.tools.pca_cov import pca_cov
from arpym.tools.logo import add_logo
from arpym.portfolio.char_portfolio import char_portfolio

## [Input parameters](https://www.arpm.co/lab/redirect.php?permalink=s_characteristic_port_rev-parameters)

In [ ]:
tau_hl = 100  # hl used to estimate (cond.) covariance of instruments
tau_hl_lam = 40  # hl used to compute the realized factor premium
n_ = 200  # number of instruments

## [Step 0](https://www.arpm.co/lab/redirect.php?permalink=s_characteristic_port_rev-implementation-step00): load data

In [ ]:
path = '~/databases/temporary-databases/'
s_mom_rk = pd.read_csv(path + 's_mom_rk.csv', index_col=0, parse_dates=True)
dates = pd.to_datetime(s_mom_rk.index).date
s_mom_rk = np.array(s_mom_rk)
path = '~/databases/global-databases/strategies/db_strategies/'
v = pd.read_csv(path + 'last_price.csv', index_col=0, parse_dates=True)
v = np.array(v)
v = v[:, :n_]
t_ = s_mom_rk.shape[0]

## [Step 1](https://www.arpm.co/lab/redirect.php?permalink=s_characteristic_port_rev-implementation-step01): reversal momentum signal

In [ ]:
s = -s_mom_rk[:, :n_]

## [Step 2](https://www.arpm.co/lab/redirect.php?permalink=s_characteristic_port_rev-implementation-step02): (conditioned) covariance matrix of risk drivers

In [ ]:
sig2_pl = np.zeros((t_, n_, n_))
sig_vol_hat_x = np.zeros((t_, n_))

for t in np.arange(0, t_):
    w_shift = 252*2  # rolling window
    # shrinked (conditioned) covariance matrix of risk drivers
    epsi = np.diff(np.log(v[t:t + w_shift, :]), axis=0)
    _, sig2_hat_x = ewm_meancov(epsi, tau_hl)
    sig_vol_hat_x[t, :] = np.sqrt(np.diag(sig2_hat_x))

    # (conditioned) covariance matrix of P&L via Taylor approximation
    delta = np.diag(v[t + w_shift, :])
    sig2_pl[t, :, :] = delta @ sig2_hat_x @ delta.T

## [Step 3](https://www.arpm.co/lab/redirect.php?permalink=s_characteristic_port_rev-implementation-step03): signal characteristics

In [ ]:
beta = np.zeros((t_, n_))

for t in np.arange(0, t_):
    beta[t, :] = v[t + w_shift, :] * sig_vol_hat_x[t, :] * \
                          s[t, :]

## [Step 4](https://www.arpm.co/lab/redirect.php?permalink=s_characteristic_port_rev-implementation-step04): characteristic portfolio and realized portfolio P&L

In [ ]:
h_char = np.zeros((t_, n_))

for t in np.arange(0, t_):
    h_char[t, :] = char_portfolio(beta[[t], :].T, sig2_pl[t, :, :]).T

## [Step 5](https://www.arpm.co/lab/redirect.php?permalink=s_characteristic_port_rev-implementation-step05): realized characteristics portfolio P&L and its st dev

In [ ]:
z_char = np.zeros(t_)
sig2_pl_h_real = np.zeros(t_)
pl_real = v[1:, :] - v[:-1, :]

for t in np.arange(0, t_):
    z_char[t] = h_char[t, :].T @ \
        pl_real[t - 1, :]
    sig2_pl_h_real[t] = h_char[t, :].T @ \
        sig2_pl[t, :, :] @ \
        h_char[t, :]

## [Step 6](https://www.arpm.co/lab/redirect.php?permalink=s_characteristic_port_rev-implementation-step06): realized factor premium

In [ ]:
lambda_hat = np.zeros(t_)
for t in range(0, t_):
    lambda_hat[t], _ = ewm_meancov(z_char[:t + 1], tau_hl_lam)

## [Step 7](https://www.arpm.co/lab/redirect.php?permalink=s_characteristic_port_rev-implementation-step07): return vs expected returns, symm. regression line

In [ ]:
exp_ret = lambda_hat[-1] * beta[-1, :] / v[-2, :]
real_ret = pl_real[-1, :] / v[-2, :]

# symmetric regression
exp_real = np.concatenate((exp_ret.reshape(-1, 1), real_ret.reshape(-1, 1)),
                          axis=1)
mu_exp_real, sig2_exp_real = meancov_sp(exp_real)
e, l = pca_cov(sig2_exp_real)
mu_real = mu_exp_real[1]
mu_exp = mu_exp_real[0]
beta_sym = -e[1, 1] / e[0, 1]
alpha_sym = mu_exp - beta_sym*mu_real
x = 2 * np.arange(-10, 11) / 10
y = beta_sym * x + alpha_sym

## Save characteristics portfolios

In [ ]:
output = {'w_shift': pd.Series(w_shift),
          'h_char': pd.Series(h_char.reshape(t_ * n_)),
          'n_': pd.Series(n_)
          }
df = pd.DataFrame(output)
df.to_csv('~/databases/temporary-databases/db_char_port.csv',
          index=None)

## Plots

In [ ]:
grid_dates = np.round(np.linspace(0, len(dates)-1, 5))

plt.style.use('arpm')

# P&L plot
fig, ax1 = plt.subplots(1, 1)
ax2 = ax1.twinx()
ax1.set_ylabel('Cum P&L', color='b')
ax2.set_ylabel('P&L', color='r')
ax2.scatter(dates, z_char, color='r', linewidth=0, marker='.')
ax2.plot(dates, np.mean(z_char) + 2 * np.sqrt(sig2_pl_h_real), color='k')
ax2.plot(dates, np.mean(z_char) - 2 * np.sqrt(sig2_pl_h_real), color='k')

# cumulative P&L
cumpl = np.cumsum(z_char)
ax1.plot(dates, np.cumsum(z_char), color='b', lw=1.5)
ax1.axis([min(dates), max(dates), np.min(cumpl), np.max(cumpl)])
plt.title('Characteristic portfolio')
plt.xticks(dates[grid_dates.astype(int)])
fmt = mdates.DateFormatter('%d-%b-%y')
plt.gca().xaxis.set_major_formatter(fmt)

add_logo(fig)
plt.tight_layout()

# expected returns vs realized returns
max_abs_ret = max(abs(np.percentile(real_ret, 100 * 0.05)),
                  abs(np.percentile(real_ret, 100 * 0.95)))

fig = plt.figure()
plt.plot(exp_ret, real_ret, 'b.')
plt.axis([np.min(exp_ret), np.max(exp_ret), -max_abs_ret, max_abs_ret])
plt.plot(x, y, 'r')
plt.xlabel('Expected returns')
plt.ylabel('Realized returns')

add_logo(fig)
plt.tight_layout()

# signal
yy = np.linspace(1, n_, 5)
stock_tick = np.round(yy)
signal, idx = np.sort(s[-1, :]), np.argsort(s[-1, :])
max_abs_signal = max(abs(min(signal)), abs(max(signal)))

fig = plt.figure()
plt.bar(range(1, len(signal)+1), signal)
plt.ylabel('Signals')
plt.axis([0, exp_ret.shape[0] + 1, -max_abs_signal, max_abs_signal])
plt.xticks(stock_tick)

add_logo(fig)
plt.tight_layout()

# exposures plot (sorted wrt the signals)
dollar_wghts = h_char[-1] * v[-2, :]

max_abs_dw = max(abs(min(dollar_wghts)), abs(max(dollar_wghts)))

fig = plt.figure()
plt.bar(range(1, len(dollar_wghts[idx])+1), dollar_wghts[idx])
plt.ylabel('Dollar weights')
plt.axis([0, exp_ret.shape[0] + 1, -max_abs_dw, max_abs_dw])
plt.xticks(stock_tick)

add_logo(fig)
plt.tight_layout()

# premium
fig = plt.figure()
plt.plot(dates, lambda_hat, color='b')
plt.axis([min(dates), max(dates), np.nanmin(lambda_hat),
          np.nanmax(lambda_hat)])
plt.ylabel('Factor premium')
plt.xticks(dates[grid_dates.astype(int)])
myFmt = mdates.DateFormatter('%d-%b-%y')
plt.gca().xaxis.set_major_formatter(myFmt)

add_logo(fig)
plt.tight_layout()